In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shutil import move,rmtree
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import os
import sys

In [ ]:
# Creating a directory structure for dataset to use keras API
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/','test/']
labeldirs = ['dogs/','cats/']
for subdir in subdirs:
    for labeldir in labeldirs:
        newdir = dataset_home+subdir+labeldir
        os.makedirs(newdir,exist_ok=True)

In [ ]:
# Delete the corrrupt Images from dataset
dataset_dirs = ['PetImages/Dog/','PetImages/Cat/']  # Replace with the path to your dataset directory

corrupted_images = []  # List to store the paths of corrupted images

# Iterate through the dataset directory
for dataset_dir in dataset_dirs:
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Attempt to open the image
                Image.open(file_path)
            except (IOError, OSError):
                # Add the file path to the list of corrupted images
                corrupted_images.append(file_path)

# Print the corrupted images
print("Corrupted Images:")
for img_path in corrupted_images:
    print(img_path)

# Delete the corrupted images
for img_path in corrupted_images:
    os.remove(img_path)

print("Corrupted images removed.")

In [ ]:
# Move the dataset Images to our created directory structure
testRatio = 0.2 # The ratio of images that will be included in test dataset
np.random.seed(1) #Seeding
src_folders = ['PetImages/Dog/','PetImages/Cat/']
for folder in src_folders:
    for file in os.listdir(folder):
        src = folder+file
        dst_dir = 'train/'
        if np.random.random()<testRatio:
            dst_dir = 'test/'
        dst = dataset_home+dst_dir
        if folder=='PetImages/Cat/':
            dst+='cats/'
        else:
            dst+='dogs/'
        dst+=file
        move(src,dst)

In [ ]:
# Defining the model
def define_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=(200,200,3)))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
    model.add(Dense(1,activation='sigmoid'))
    # compile model
    opt= SGD(learning_rate=0.001,momentum=0.9)
    model.compile(opt,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [ ]:

model = define_model()
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow_from_directory(directory='dataset_dogs_vs_cats/train/',
                                       class_mode='binary',batch_size=64,target_size=(200,200))
test_it = datagen.flow_from_directory(directory='dataset_dogs_vs_cats/test/',
                                      class_mode='binary',batch_size=64,target_size=(200,200))
# training the model
history = model.fit(train_it,steps_per_epoch=len(train_it),
                    validation_data=test_it,validation_steps=len(test_it),epochs=20,verbose=0)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))
# plot diagnostic learning curves
# plot loss
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
# save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.close()